In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SimpleRNN, SpatialDropout1D

from sklearn.model_selection import train_test_split
from sklearn import metrics

Ancak, Metin gösterimi adımına geçmeden önce, önce ön işleme tabi tutulması gereken temizlenmiş bir veri kümesi elde etmeliyiz. 

## Veri kümesini yükleme ve temizleme

Veri kümesi olarak, [Twitter US Airline Sentiment](https://www.kaggle.com/crowdflower/twitter-airline-sentiment) veri kümesini kullanacağız.

In [ ]:
DATA_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining/main/PART2/Day14-NLP/notebooks/datasets/"

DATASET_PATH = DATA_DIR + "twitter_airline_sentiment_tweets.csv"

In [ ]:
df = pd.read_csv(DATASET_PATH)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
sns.countplot(x = "airline_sentiment", data = df)

## Temel Ön İşlemler

1. StopWord'lerin temizlenmesi
2. Küçük harfe çevirme
3. Stemming (Kök alma)
4. Tokenization


In [ ]:
data = df[["airline_sentiment", "text"]]
data

In [ ]:
def sentiment(x):
    if x == 'positive':
        return 1
    elif x == 'negative':
        return -1
    else:
        return 0

In [ ]:
data["airline_sentiment_label"] = data["airline_sentiment"].apply(sentiment)
data

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stopwords = stopwords.words('english')

def clean_the_tweet(text):
  tokens= nltk.word_tokenize(re.sub("[^a-zA-Z]", " ",text))
  tokens = [token.lower() for token in tokens if token not in stopwords]
  return ' '.join(tokens[2:])

In [ ]:
print(f"Orignal Text : {data.text[11]}")
print()
print(f"Preprocessed Text : {clean_the_tweet(data.text[11])}")

In [ ]:
data["clean_text"] = data.text.map(clean_the_tweet)
data.head()

In [ ]:
data.iloc[10].text

In [ ]:
data.iloc[10].clean_text

In [ ]:
data

In [ ]:
max_features = 2000
token = Tokenizer(num_words=max_features, split = ' ')
token.fit_on_texts(data['clean_text'].values)

X = token.texts_to_sequences(data['clean_text'].values)
X = pad_sequences(X)

In [ ]:
X.shape


In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
import tensorflow as tf

tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
Y = pd.get_dummies(data['airline_sentiment_label']).values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, random_state=42)

In [ ]:
batch_size = 32
history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_data=(X_test, y_test), verbose=2)


In [ ]:
def plot_training_hist(history):
    '''Function to plot history for accuracy and loss'''
    
    fig, ax = plt.subplots(1, 2, figsize=(10,4))
    # first plot
    ax[0].plot(history.history['accuracy'])
    ax[0].plot(history.history['val_accuracy'])
    ax[0].set_title('Model Accuracy')
    ax[0].set_xlabel('epoch')
    ax[0].set_ylabel('accuracy')
    ax[0].legend(['train', 'validation'], loc='best')
    # second plot
    ax[1].plot(history.history['loss'])
    ax[1].plot(history.history['val_loss'])
    ax[1].set_title('Model Loss')
    ax[1].set_xlabel('epoch')
    ax[1].set_ylabel('loss')
    ax[1].legend(['train', 'validation'], loc='best')
    
plot_training_hist(history)

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(model, X_test, y_test):
    '''Function to plot confusion matrix for the passed model and the data'''
    
    sentiment_classes = ['Negative', 'Neutral', 'Positive']
    # use model to do the prediction
    y_pred = model.predict(X_test)
    # compute confusion matrix
    cm = confusion_matrix(np.argmax(np.array(y_test),axis=1), np.argmax(y_pred, axis=1))
    # plot confusion matrix
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, cmap=plt.cm.Blues, annot=True, fmt='d', 
                xticklabels=sentiment_classes,
                yticklabels=sentiment_classes)
    plt.title('Confusion matrix', fontsize=16)
    plt.xlabel('Actual label', fontsize=12)
    plt.ylabel('Predicted label', fontsize=12)
    
plot_confusion_matrix(model, X_test, y_test)

In [ ]:
# score = model.predict(X_test)
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=2)
print('score', score)
print('accuracy', acc)

In [ ]:
#text = ['i would recommend it if you have no other options']
#text = ['the food was meh']
text = ['I hate when I have to call and wake people up']

text = token.texts_to_sequences(text)
text = pad_sequences(text, maxlen=22, dtype='int32', value=0)
res = model.predict(text, batch_size=1,verbose = 2)
res



In [ ]:
if np.argmax(res[0]) == 0:
    print("Negetive Comment")
elif np.argmax(res[0]) == 1:
    print("Posetive Comment")
else:
  print("Neutral Comment")